In [1]:
%reset
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../")
import torch
import numpy as np
import defmod as dm
import multimodule_usefulfunctions as mm 
import matplotlib.pyplot as plt

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
nb_pts1 = 10
nb_pts2 = 7
dim = 2

o1 = torch.tensor([1,1]).float()
o2 = torch.tensor([5,5]).float()
origin = [o1,o2]
radius = [2,1]
numberPoints = [nb_pts1, nb_pts2]
gd_list = mm.multipleCircles(origin, radius, numberPoints)

In [3]:
plt.scatter(gd_list[0][:, 0].detach().numpy(), gd_list[0][:, 1].detach().numpy(), c='red')
plt.scatter(gd_list[1][:, 0].detach().numpy(), gd_list[1][:, 1].detach().numpy(), c='blue')

plt.axis('equal')
#plt.show()

(-1.3613887604218857,
 6.3920313523721441,
 -1.263359584052745,
 6.3805314957338002)

In [4]:
# Defining the deformation modules
sigma1 = 10
manifold1 = dm.manifold.Landmarks(dim, nb_pts1, gd=gd_list[0].view(-1))
trans1 = dm.deformationmodules.Translations(manifold1, sigma1)

sigma2 = 10
manifold2 = dm.manifold.Landmarks(dim, nb_pts2, gd=gd_list[1].view(-1))
trans2 = dm.deformationmodules.Translations(manifold2, sigma2)

module_list = [trans1, trans2]
sigma_bg = 1
modules = dm.multishape.MultiShapeModule(module_list, sigma_bg)


In [5]:
# Setting cotangent variables
p1 = torch.cat([torch.ones(nb_pts1,1, requires_grad=True), torch.zeros(nb_pts1,1)],1).view(-1)
p2 = torch.cat([-torch.ones(nb_pts2,1,requires_grad=True), torch.zeros(nb_pts2,1)],1).view(-1)
cotan2 = [p1, p2, [p1, p2]]
cotan = [torch.rand(nb_pts1*dim, requires_grad=True), torch.rand(nb_pts2*dim, requires_grad=True), [torch.rand((nb_pts1)*dim, requires_grad=True),torch.rand((nb_pts2)*dim, requires_grad=True)]]
modules.manifold.fill_cotan(cotan2)



In [6]:
constr_Id = dm.constraints.Identity()
l, h = modules.compute_geodesic_variables(modules.manifold, constr_Id)

In [7]:
#constr_Id(modules)

In [8]:
# Define Hamiltonian function
#H = dm.hamiltonian_multishape.Hamiltonian_multi(modules, constr_Id)

In [9]:
import defmod.hamiltonian_multishape as hamiltonian
H = hamiltonian.Hamiltonian_multi(modules, constr_Id)

In [10]:
# Check derivative of Hamiltonian wrt controls is zero
def controlslist2tensor(controls_list):
    return torch.cat([controls_list[0].view(-1,1),controls_list[1].view(-1,1), controls_list[-1][0].view(-1,1),controls_list[-1][1].view(-1,1)])

def controlstensor2list(controls, nb_pts, dim):
    n = sum(nb_pts)
    return [controls[:dim*nb_pts[0]], controls[dim*nb_pts[0]:dim*(nb_pts[0]+nb_pts[1])], [controls[dim*n:dim*(n+nb_pts[0])], controls[dim*(n+nb_pts[0]):]]]


def Hamiltonian(controls):
    #modules.controls = controlstensor2list(controls, numberPoints, dim)
    modules.fill_controls(controls)
    H = hamiltonian.Hamiltonian_multi(modules, constr_Id)
    return H()


controls = dm.usefulfunctions.flatten_tensor_list(modules.controls, [])


a = torch.autograd.grad(Hamiltonian(modules.controls), controls, retain_graph=True)
len(a)

[sum(torch.abs(ai)) for ai in a]

[tensor(1.5259e-05),
 tensor(2.3842e-06),
 tensor([3.5763e-06]),
 tensor([1.4305e-06])]

In [11]:
H.geodesic_controls()

In [12]:
def applymom(l):
    #H.module.manifold[0].fill_gd(l[0])
    #H.module.manifold[1].fill_gd(l[1])
    #H.module.manifold[2].fill_gd([l[2], l[3]])
    H.module.manifold[0].fill_cotan(l[0])
    H.module.manifold[1].fill_cotan(l[1])
    H.module.manifold[2].fill_cotan([l[2], l[3]])
    H.geodesic_controls()
    #print([H.module[i].manifold.inner_prod_field(H.module[i].field_generator()) for i in range(3)])
    
    return sum([H.module.manifold.manifold_list[i].inner_prod_field(H.module[i].field_generator()) for i in range(3)])

#sum([H.module[i].manifold.inner_prod_module(H.module[i]) for i in range(1)])
    #return H.apply_mom()

In [13]:
def Hamiltonian_p(cotan):
    
    cotan = controlstensor2list(cotan, [nb_pts1, nb_pts2], dim)
    #H.module.manifold.fill_cotan(cotan)
    
    H.module.manifold[0].fill_cotan(cotan[0])
    H.module.manifold[1].fill_cotan(cotan[1])
    H.module.manifold[2].fill_cotan(cotan[2])
    
    H.geodesic_controls()
    return H()

In [14]:
cotan = controlslist2tensor(H.module.manifold.cotan)
deltaH = torch.autograd.grad(Hamiltonian_p(cotan), cotan, create_graph=True)

In [15]:
def Hamiltonian(l):
    H.module.manifold[0].fill_cotan(l[0])
    H.module.manifold[1].fill_cotan(l[1])
    H.module.manifold[2].fill_cotan([l[2], l[3]])
    H.geodesic_controls()
    return H()

In [16]:
def costterm(l):
    H.module.manifold[0].fill_cotan(l[0])
    H.module.manifold[1].fill_cotan(l[1])
    H.module.manifold[2].fill_cotan([l[2], l[3]])
    H.geodesic_controls()
    return sum([H.module[i].cost() for i in range(3)])

In [17]:
def constraintsterm(l):
    H.module.manifold[0].fill_cotan(l[0])
    H.module.manifold[1].fill_cotan(l[1])
    H.module.manifold[2].fill_cotan([l[2], l[3]])
    H.geodesic_controls()
    return H.apply_constr()

In [18]:
def controlslist2tensor(controls_list):
    return torch.cat([modules.controls[0].view(-1,1),modules.controls[1].view(-1,1), modules.controls[-1][0].view(-1,1),modules.controls[-1][1].view(-1,1)])


def geodesiccontrols(l):
    H.module.manifold[0].fill_cotan(l[0])
    H.module.manifold[1].fill_cotan(l[1])
    H.module.manifold[2].fill_cotan([l[2], l[3]])    
    H.geodesic_controls()

    return controlslist2tensor(H.module.controls)



In [19]:
cotan = [*H.module.manifold.unroll_cotan()]

deltaapplymom = torch.autograd.grad(applymom(cotan), cotan, create_graph=True)
deltaH = torch.autograd.grad(Hamiltonian(cotan), [cotan[0], cotan[1], cotan[2], cotan[3]], create_graph=True)
deltacost = torch.autograd.grad(costterm(cotan), [cotan[0], cotan[1], cotan[2], cotan[3]], create_graph=True)
deltaconstr = torch.autograd.grad(constraintsterm(cotan), [cotan[0], cotan[1], cotan[2], cotan[3]], create_graph=True)


In [20]:
print(deltaH)

(tensor([[3.3984],
        [0.0000],
        [3.3980],
        [0.0000],
        [3.3982],
        [0.0000],
        [3.3989],
        [0.0000],
        [3.3998],
        [0.0000],
        [3.4007],
        [0.0000],
        [3.4011],
        [0.0000],
        [3.4009],
        [0.0000],
        [3.4003],
        [0.0000],
        [3.3993],
        [0.0000]], grad_fn=<AddBackward0>), tensor([[-4.4250],
        [ 0.0000],
        [-4.4251],
        [ 0.0000],
        [-4.4249],
        [ 0.0000],
        [-4.4244],
        [ 0.0000],
        [-4.4240],
        [ 0.0000],
        [-4.4241],
        [ 0.0000],
        [-4.4245],
        [ 0.0000]], grad_fn=<AddBackward0>), tensor([[3.3984],
        [0.0000],
        [3.3980],
        [0.0000],
        [3.3982],
        [0.0000],
        [3.3989],
        [0.0000],
        [3.3998],
        [0.0000],
        [3.4007],
        [0.0000],
        [3.4011],
        [0.0000],
        [3.4009],
        [0.0000],
        [3.4003],
        [0.0000

In [23]:
print(deltaapplymom)

(tensor([[6.7969],
        [0.0000],
        [6.7960],
        [0.0000],
        [6.7963],
        [0.0000],
        [6.7977],
        [0.0000],
        [6.7997],
        [0.0000],
        [6.8013],
        [0.0000],
        [6.8022],
        [0.0000],
        [6.8019],
        [0.0000],
        [6.8005],
        [0.0000],
        [6.7986],
        [0.0000]], grad_fn=<AddBackward0>), tensor([[-8.8499],
        [ 0.0000],
        [-8.8502],
        [ 0.0000],
        [-8.8497],
        [ 0.0000],
        [-8.8487],
        [ 0.0000],
        [-8.8480],
        [ 0.0000],
        [-8.8481],
        [ 0.0000],
        [-8.8490],
        [ 0.0000]], grad_fn=<AddBackward0>), tensor([[6.7969],
        [0.0000],
        [6.7960],
        [0.0000],
        [6.7963],
        [0.0000],
        [6.7977],
        [0.0000],
        [6.7997],
        [0.0000],
        [6.8013],
        [0.0000],
        [6.8022],
        [0.0000],
        [6.8019],
        [0.0000],
        [6.8005],
        [0.0000

In [24]:
print(deltacost)

(tensor([[3.3984],
        [0.0000],
        [3.3980],
        [0.0000],
        [3.3982],
        [0.0000],
        [3.3989],
        [0.0000],
        [3.3998],
        [0.0000],
        [3.4007],
        [0.0000],
        [3.4011],
        [0.0000],
        [3.4009],
        [0.0000],
        [3.4003],
        [0.0000],
        [3.3993],
        [0.0000]], grad_fn=<AddBackward0>), tensor([[-4.4250],
        [ 0.0000],
        [-4.4251],
        [ 0.0000],
        [-4.4249],
        [ 0.0000],
        [-4.4244],
        [ 0.0000],
        [-4.4240],
        [ 0.0000],
        [-4.4241],
        [ 0.0000],
        [-4.4245],
        [ 0.0000]], grad_fn=<AddBackward0>), tensor([[3.3984],
        [0.0000],
        [3.3980],
        [0.0000],
        [3.3982],
        [0.0000],
        [3.3989],
        [0.0000],
        [3.3998],
        [0.0000],
        [3.4007],
        [0.0000],
        [3.4011],
        [0.0000],
        [3.4009],
        [0.0000],
        [3.4003],
        [0.0000

In [25]:
print(deltaconstr)

(tensor([[ 4.7684e-07],
        [ 0.0000e+00],
        [ 1.4305e-06],
        [ 0.0000e+00],
        [ 3.8147e-06],
        [ 0.0000e+00],
        [ 9.5367e-07],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 2.8610e-06],
        [ 0.0000e+00],
        [-2.3842e-06],
        [ 0.0000e+00],
        [ 2.3842e-06],
        [ 0.0000e+00],
        [ 1.4305e-06],
        [ 0.0000e+00],
        [ 1.9073e-06],
        [ 0.0000e+00]], grad_fn=<AddBackward0>), tensor([[2.3842e-07],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [2.3842e-07],
        [0.0000e+00],
        [2.3842e-07],
        [0.0000e+00],
        [2.3842e-07],
        [0.0000e+00],
        [2.3842e-07],
        [0.0000e+00],
        [2.3842e-07],
        [0.0000e+00]], grad_fn=<AddBackward0>), tensor([[-3.5763e-07],
        [ 0.0000e+00],
        [ 2.3842e-07],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [-2.3842e-07],
        [ 0.0000e+

In [26]:
# action 
speed_action = [gdi.action(modulei).tan for gdi, modulei in zip(H.module.manifold.manifold_list, H.module)] 

print(speed_action)
H.module.manifold.fill_tan(speed_action)

sum([H.module.manifold.manifold_list[i].inner_prod_field(H.module.module_list[i].field_generator()) for i in range(3)])

[tensor([3.3984, 0.0000, 3.3980, 0.0000, 3.3981, 0.0000, 3.3989, 0.0000, 3.3998,
        0.0000, 3.4007, 0.0000, 3.4011, 0.0000, 3.4009, 0.0000, 3.4003, 0.0000,
        3.3993, 0.0000], grad_fn=<ViewBackward>), tensor([-4.4250,  0.0000, -4.4251,  0.0000, -4.4249,  0.0000, -4.4244,  0.0000,
        -4.4240,  0.0000, -4.4241,  0.0000, -4.4245,  0.0000],
       grad_fn=<ViewBackward>), [tensor([3.3984, 0.0000, 3.3980, 0.0000, 3.3981, 0.0000, 3.3989, 0.0000, 3.3998,
        0.0000, 3.4007, 0.0000, 3.4011, 0.0000, 3.4009, 0.0000, 3.4003, 0.0000,
        3.3993, 0.0000], grad_fn=<ViewBackward>), tensor([-4.4250,  0.0000, -4.4251,  0.0000, -4.4249,  0.0000, -4.4244,  0.0000,
        -4.4240,  0.0000, -4.4241,  0.0000, -4.4245,  0.0000],
       grad_fn=<ViewBackward>)]]


tensor(129.9349, grad_fn=<AddBackward0>)

In [27]:
fields = modules.field_generator()
field_bg = fields[-1]
i=0
gd_bg = modules.manifold.manifold_list[-1].manifold_list[i].gd

print(fields[i](modules.manifold.gd[i].view(-1,modules.manifold.dim)))
print(field_bg(gd_bg.view(-1,modules.manifold.dim)))

modules.module_list[i].field_generator()(modules.manifold.gd[i].view(-1,modules.manifold.dim))
modules.module_list[-1].field_generator()(modules.manifold.gd[i].view(-1,modules.manifold.dim))

gd_bg, modules.manifold.manifold_list[0].gd
modules.module_list[i].controls, modules.module_list[-1].controls
Bg = dm.deformationmodules.Background(modules.module_list[:-1], sigma_bg)
Bg.fill_controls(modules.module_list[-1].controls)
Bg.field_generator()(modules.manifold.gd[i].view(-1,modules.manifold.dim))

tensor([[3.3984, 0.0000],
        [3.3980, 0.0000],
        [3.3981, 0.0000],
        [3.3989, 0.0000],
        [3.3998, 0.0000],
        [3.4007, 0.0000],
        [3.4011, 0.0000],
        [3.4009, 0.0000],
        [3.4003, 0.0000],
        [3.3993, 0.0000]], grad_fn=<AsStridedBackward>)
tensor([[3.3984, 0.0000],
        [3.3980, 0.0000],
        [3.3981, 0.0000],
        [3.3989, 0.0000],
        [3.3998, 0.0000],
        [3.4007, 0.0000],
        [3.4011, 0.0000],
        [3.4009, 0.0000],
        [3.4003, 0.0000],
        [3.3993, 0.0000]], grad_fn=<AddBackward0>)


tensor([[3.3984, 0.0000],
        [3.3980, 0.0000],
        [3.3981, 0.0000],
        [3.3989, 0.0000],
        [3.3998, 0.0000],
        [3.4007, 0.0000],
        [3.4011, 0.0000],
        [3.4009, 0.0000],
        [3.4003, 0.0000],
        [3.3993, 0.0000]], grad_fn=<AddBackward0>)

In [ ]:
manifold_t, modules_t = dm.shooting.shoot_euler(H)

In [ ]:
gd_t = [man.gd for man in manifold_t]


In [ ]:
import matplotlib.cm as cm

source = gd_list

# Geometrical descriptors at initial time
plt.subplot(1,3,1)
plt.scatter(source[0][:, 0].detach().numpy(), source[0][:, 1].detach().numpy(), c='red')
plt.scatter(source[1][:, 0].detach().numpy(), source[1][:, 1].detach().numpy(), c='blue')
plt.axis('equal')

# Geometrical Descriptors transported over time by shooting
plt.subplot(1,3,2)
colors = cm.rainbow(np.linspace(0, 1, len(gd_t)))
for i,c in zip(range(len(gd_t)), colors):
    plt.scatter(gd_t[i][0].view(-1,dim)[:,0].detach().numpy(), gd_t[i][0].view(-1,dim)[:, 1].detach().numpy(), color=c)
    plt.scatter(gd_t[i][1].view(-1,dim)[:,0].detach().numpy(), gd_t[i][1].view(-1,dim)[:, 1].detach().numpy(), color=c)
plt.axis('equal')

# Geometrical descriptors at final time
plt.subplot(1,3,3)
plt.scatter(gd_t[-1][0].view(-1,dim)[:, 0].detach().numpy(), gd_t[-1][0].view(-1,dim)[:, 1].detach().numpy(), c='red')
plt.scatter(gd_t[-1][1].view(-1,dim)[:, 0].detach().numpy(), gd_t[-1][1].view(-1,dim)[:, 1].detach().numpy(), c='blue')
plt.axis('equal')



for i in range(len(gd_t)):
    plt.figure()
    plt.scatter(gd_t[i][0].view(-1,dim)[:, 0].detach().numpy(), gd_t[i][0].view(-1,dim)[:, 1].detach().numpy(), c='red')
    plt.scatter(gd_t[i][1].view(-1,dim)[:, 0].detach().numpy(), gd_t[i][1].view(-1,dim)[:, 1].detach().numpy(), c='blue')
    plt.axis([-5,20,-5,20])

plt.show()

In [ ]:
gd_t[0][0], gd_list[0]

In [ ]:
constr_t = []
for i in range(len(manifold_t)):
    modules.compute_geodesic_variables(manifold_t[i], constr_Id)
    print(modules.controls)
    print(torch.norm(constr_Id(modules)))
    constr_t = [*constr_t, constr_Id(modules)]

constr_t[0]-constr_t[5]